In [257]:
from bs4 import BeautifulSoup
import urllib.request
import re

#write collected data to .csv file as a pandas dataframe
import numpy
import pandas as pd 

#extract price
def price_from_json(flat):
    price = flat.find("div", {"class": "popup-prices"})['data-prices']
    price_ind = price.find('RUB":')
    price_cut = price[price_ind + 5:]
    price_in_rub = price_cut.split(",")[0]
    return int(price_in_rub)

In [ ]:
#url to first pages
yandex_real_estate = urllib.request.urlopen("https://realty.yandex.ru/sankt-peterburg/kupit/kvartira/").read()
url = "https://www.avito.ru/sankt-peterburg/kvartiry?s_trg=1"
avito_real_estate = urllib.request.urlopen("https://www.avito.ru/sankt-peterburg/kvartiry/prodam").read()

In [ ]:
#parse avito with beautifoul soup
avito_soup = BeautifulSoup(avito_real_estate)
url_base = "https://www.avito.ru"
flats = avito_soup.findAll("div", {"class":"item_table-description"})
page_count = 0

data_flats = parse_page(flats, [])

while True:
    try:
        page_count += 1
        print("Page: ", page_count)
        next_page = avito_soup.find("a", {"class":"pagination-page"}).get('href')
        flats = avito_soup.findAll("div", {"class":"item_table-description"})
        data_flats = parse_page_avito(flats, data_flats)
        
        if page_count > 100:
                break
    except Error:
        break

print("finish")
print("data len: ", len(data_flats))



In [230]:
#extract all relevant data from single page
def parse_page_avito(flats, data_flats):
    rent_count = 0
    print("flats on page: ", len(flats))
    for (index, flat) in enumerate(flats):

        print("Flat number", index)

        #price
        print("price:")
        price = price_from_json(flat)
        if(price < 200_000):
            rent_count += 1
            print("Rent found, skip")
            continue
        print(price)
        print()


        #rooms, area, floor level
        print("rooms:")
        description = flat.find("a", {"class": "item-description-title-link"})['title']
        try_rooms = description.split(",")[0][0]
        if try_rooms.isnumeric():
            rooms = int(try_rooms)
        else:
            rooms = 1
        print(rooms)
        print()

        print("area:")
        area = float(description.split(",")[1].split(" ")[1])
        print(area, "m^2")
        print()

        print(description)

        print("floor:")
        floor = description.split(",")[2].split(" ")[1].split("/")[0]
        floors_total = description.split(",")[2].split(" ")[1].split("/")[1]
        print("" + floor + ", total floors: " + floors_total)
        print()

        #subway, distance to subway, adress
        full_adress = flat.find("p", {"class": "address"}).getText()

        adress = ','.join(full_adress.split(",")[1:])
        print("adress:")
        print(adress)
        print()

        subway = full_adress.split(",")[0]
        for (ind,el) in enumerate(subway.split()):
            try:
                float(el)
                subway_station = ' '.join(subway.split()[:ind])
                distance = float(subway.split()[ind])
            except ValueError:
                pass

        print("subway:")
        print(subway_station)
        print()

        if distance < 99:
            distance *= 1000
        print("distance to subway:")
        print(distance, "meters")
        print()

        print("go to next page")
        data_flats.append((price, rooms, area, floor, floors_total, adress, subway_station, distance))
    
    print(rent_count)
    return data_flats
    

In [202]:
#flat data class combining all relevant data
class Flat:
    
    def __init__(self, flat_price, flat_rooms, flat_area, flat_floor, 
                 flat_floors_total, flat_address, flat_subway_st, flat_distance_to_subway):
        self.flat_price = flat_price
        self.flat_rooms = flat_rooms 
        self.flat_area = flat_area
        self.flat_floor = flat_floor
        self.flat_floors_total = flat_floors_total
        self.flat_address = flat_address
        self.flat_subway_st = flat_subway_st
        self.flat_distance_to_subway = flat_distance_to_subway
        
    def __str__(self):
        return ''.join("adress = "+ flat_address + 
                       ", price = "+ str(flat_price)+ 
                       ", rooms = "+ str(flat_rooms)+
                        ", floor = "+ str(flat_floor)+ 
                       ", area = " + str(flat_area) +
                      ", total_floors = "+ str(flat_floors_total)+ 
                       ", subway = "+ flat_subway_st + 
                       ", distance to subway = " + flat_distance_to_subway)

In [220]:
#
subways = set()
for flat in data_flats:
    subways.add(flat[6])
print(subways)

{'Чкаловская', 'Проспект Просвещения', 'Пионерская', 'Пролетарская', 'Приморская', 'Купчино', 'Проспект Большевиков', 'Елизаровская', 'Ломоносовская', 'Комендантский проспект', 'Василеостровская', 'Ладожская', 'Девяткино', 'Спортивная', 'Московская', 'Чернышевская', 'Автово', 'Звездная', 'Обухово', 'Парнас', 'Достоевская', 'Проспект Ветеранов', 'Новочеркасская', 'Гражданский проспект'}


In [225]:
#clean up parsed data
new_data_flats = []
for flat in data_flats:
    new_flat = (flat[0],flat[1],flat[2],flat[3],flat[4],flat[5].replace('\xa0', '', 2),flat[6],flat[7])
    new_data_flats.append(new_flat)

In [229]:
#write collected data to .csv file as a pandas dataframe
import numpy
import pandas as pd 

a = numpy.asarray(new_data_flats)
pd.DataFrame(a).to_csv("data.csv", header=['price', 'rooms', 'area', 'floor', 'floors_total', 'adress',
                                           'subway_station', 'distance to subway'])

 Парс сайта cian.ru

In [154]:
#read page, load to soup object
cian_real_estate = urllib.request.urlopen("https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1").read()
cian_soup = BeautifulSoup(cian_real_estate)

In [258]:
def next_page(first_page, soup):
    next_p = soup.find("li", {"class": re.compile('^.*list-item--active.*')})
    if next_p and next_p.nextSibling:
        return next_p.nextSibling.a['href']
    else:
        return -1

def cian_flats(cian_soup, data_flats):
    counter = 0
    flats = cian_soup.findAll("div", {"class": "c6e8ba5398--info-container--YhZ2y"})
    print(len(flats))
    for flat in flats:
        
        counter += 1
        # rooms, area, floor
        addr = flat.find("div", {"class": re.compile('^.*title.*')})
        if addr:
            try:
                rooms = int(addr.text[0])
                print('rooms = ', rooms)
            except Exception:
                rooms = 1
            
            area = float(addr.text.split(',')[1].split()[0])
            print('area = ', area)
            floor = int(addr.text.split(',')[2].split(' ')[1].split('/')[0])
            print('floors = ', floor)
            floors_total = int(addr.text.split(',')[2].split(' ')[1].split('/')[1])
            print('floors total = ', floors_total)
            
            print(addr.text)

        subway = flat.find("div", {"class": re.compile('^.*underground-name.*')})
        if subway:
            subway = subway.text
            print(subway)
        else:
            subway = None

        remoteness = flat.find("div", {"class": re.compile('^.*remoteness.*')})
        if remoteness:
            print(remoteness.text)
            if remoteness.text.split('\xa0')[2] == 'пешком':
                remoteness = int(remoteness.text.split('\xa0')[0]) * 100
            else:
                remoteness = int(remoteness.text.split('\xa0')[0]) * 1000
            print('distance to subway = ',remoteness)
        else:
            remoteness = None
            
        adress = flat.findAll("a", {"class": re.compile('^.*address.*')})
        if adress:
            if(len(adress) > 5):
                adress = adress[4].text+ ', ' + adress[5].text
                print(adress)
            else:
                continue
                
        price = flat.find("div", {"class": re.compile('^.*price.*')}).find("div",{'class': re.compile('.*header.*')})
        if price:
            print(price.text)
            price_array = price.text.split(' ')[:len(price.text.split(' ')) - 1]
            price = 0
            for el in price_array:
                price *= 1000
                price += int(el)
            print('price = ', price)
            
        data_flats.append((price, rooms, area, floor, floors_total, adress, subway, remoteness))
        print()
    return data_flats

In [260]:
cian_data

[(11945000, 1, 51.0, 6, 8, 'Смольный проспект, 17', 'Чернышевская', 1000),
 (18960000, 3, 181.0, 10, 10, 'Афанасьевская улица, 1', 'Озерки', 5000),
 (10495000,
  3,
  86.0,
  10,
  11,
  'Кременчугская улица, 17к3',
  'Площадь Восстания',
  1000),
 (5650000,
  1,
  37.0,
  14,
  17,
  'Богатырский проспект, 36к1',
  'Комендантский проспект',
  1800),
 (7650000, 3, 75.0, 1, 5, 'Перевозный переулок, 9', 'Новочеркасская', 500),
 (5750000,
  1,
  43.0,
  17,
  20,
  'Парашютная улица, 38к4',
  'Комендантский проспект',
  3000),
 (18970000, 4, 256.0, 10, 10, 'Афанасьевская улица, 1', 'Удельная', 8000),
 (22750000, 5, 420.0, 10, 10, 'Афанасьевская улица, 1', 'Озерки', 2600),
 (6499000,
  1,
  43.0,
  10,
  10,
  'Приморский проспект, 54к2',
  'Старая Деревня',
  3100),
 (7730000, 3, 84.0, 1, 4, 'Пулковское тер., Неоклассика ЖК', 'Купчино', None),
 (9022000, 2, 65.0, 13, 19, 'Общественный переулок, 5', 'Елизаровская', 1000),
 (17000000, 3, 86.0, 7, 11, 'Пионерская улица, 50', 'Чкаловская', 15

In [259]:
# cian_url = "https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"
# base_url = cian_url.split('cian.ru')[0] + 'cian.ru' 
# cian_real_estate = urllib.request.urlopen(cian_url).read()
# cian_soup = BeautifulSoup(cian_real_estate)
cian_data = cian_flats(cian_soup, [])

28
rooms =  2
area =  53.0
floors =  2
floors total =  22
2-комн. кв., 53 м², 2/22 этаж
Проспект Ветеранов
20 минут на транспорте
distance to subway =  20000
rooms =  1
area =  51.0
floors =  6
floors total =  8
1-комн. кв., 51 м², 6/8 этаж
Чернышевская
10 минут пешком
distance to subway =  1000
Смольный проспект, 17
11 945 000 ₽
price =  11945000

rooms =  3
area =  85.0
floors =  6
floors total =  25
3-комн. кв., 85 м², 6/25 этаж
Комендантский проспект
30 минут пешком
distance to subway =  3000
rooms =  3
area =  181.0
floors =  10
floors total =  10
3-комн. кв., 181 м², 10/10 этаж
Озерки
5 минут на транспорте
distance to subway =  5000
Афанасьевская улица, 1
18 960 000 ₽
price =  18960000

rooms =  3
area =  86.0
floors =  10
floors total =  11
3-комн. кв., 86 м², 10/11 этаж
Площадь Восстания
10 минут пешком
distance to subway =  1000
Кременчугская улица, 17к3
10 495 000 ₽
price =  10495000

rooms =  1
area =  37.0
floors =  14
floors total =  17
1-комн. кв., 37 м², 14/17 этаж
Комен

In [198]:

cian_url = "https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"
base_url = cian_url.split('cian.ru')[0] + 'cian.ru' 
cian_real_estate = urllib.request.urlopen(cian_url).read()
cian_soup = BeautifulSoup(cian_real_estate)
counter = 0
flats = []
while counter < 59:
    counter += 1
    flats = cian_flats(cian_soup, flats)
    cian_url = next_page(cian_url, cian_soup)
    if cian_url == -1:
        break
    if base_url not in cian_url:
        cian_url = base_url + cian_url
    print("url = ", cian_url)
    cian_html = urllib.request.urlopen(cian_url).read()
    cian_soup = BeautifulSoup(cian_html)
    print("counter = ", counter)
    


28
Квартира-студия на Б. МонетнойСтудия, 21 м², 1/6 этаж
Горьковская
15 минут пешком
Большая Монетная улица, 23
2 768 000 ₽

1-комн. кв., 35 м², 6/12 этаж
Комендантский проспект
10 минут на транспорте
4 184 860 ₽

3-комн. кв., 102 м², 2/8 этаж
Удельная
10 минут пешком
Ярославский проспект, 27
16 100 000 ₽

3-комн. кв., 96 м², 12/15 этаж
Новочеркасская
10 минут пешком
14 441 970 ₽

1-комн. кв., 37 м², 14/17 этаж
Комендантский проспект
18 минут пешком
Богатырский проспект, 36к1
5 650 000 ₽

3-комн. кв., 75 м², 1/5 этаж
Новочеркасская
5 минут пешком
Перевозный переулок, 9
7 650 000 ₽

1-комн. кв., 43 м², 17/20 этаж
Комендантский проспект
30 минут пешком
Парашютная улица, 38к4
5 750 000 ₽

3-комн. кв., 86 м², 10/11 этаж
Площадь Восстания
10 минут пешком
Кременчугская улица, 17к3
10 495 000 ₽

Апартаменты-студия, 29 м², 7/9 этаж
Фрунзенская
8 минут пешком
Смоленская улица, 18
5 690 000 ₽

5-комн. кв., 420 м², 10/10 этаж
Озерки
26 минут пешком
Афанасьевская улица, 1
22 750 000 ₽

1-комн. кв.

counter =  112
28
1-комн. кв., 38 м², 22/25 этаж
Комендантский проспект
30 минут пешком
3 627 100 ₽

1-комн. кв., 38 м², 24/25 этаж
Комендантский проспект
30 минут пешком
3 644 200 ₽

1-комн. кв., 38 м², 21/25 этаж
Комендантский проспект
30 минут пешком
3 626 150 ₽

Студия, 20 м², 8/14 этаж
Выборгская
20 минут пешком
Большой Сампсониевский проспект, 68Ж
3 265 830 ₽

Студия, 21 м², 13/14 этаж
Выборгская
20 минут пешком
Большой Сампсониевский проспект, 68Ж
3 261 750 ₽

Своб. планировка, 27 м², 2/18 этаж
Приморская
10 минут на транспорте
3 295 880 ₽

Студия, 27 м², 1/4 этаж
Московская
25 минут на транспорте
Захаржевская улица, 14
3 289 440 ₽

Студия, 26 м², 18/25 этаж
Звёздная
15 минут пешком
3 283 690 ₽

1-комн. кв., 34 м², 6/24 этаж
Пролетарская
15 минут на транспорте
3 279 410 ₽

1-комн. кв., 35 м², 11/21 этаж
Ладожская
8 минут на транспорте
3 289 450 ₽

1-комн. кв., 37 м², 2/21 этаж
Ладожская
8 минут на транспорте
3 295 980 ₽

1-комн. кв., 39 м², 1/15 этаж
Московская
8 минут на трансп

counter =  221
28
2-комн. кв., 63 м², 12/25 этаж
Комендантский проспект
5 минут пешком
Комендантский проспект, 17к1
8 900 000 ₽

5-комн. кв., 167 м², 8/11 этаж
Петроградская
15 минут пешком
улица Грота, 1-3В
46 760 000 ₽

1-комн. кв., 38 м², 3/8 этаж
Площадь Восстания
20 минут пешком
10-я Советская улица, 8а
7 914 660 ₽

1-комн. кв., 39 м², 2/26 этаж
Парнас
6 минут пешком
улица Михаила Дудина, 25к1
3 999 999 ₽

1-комн. кв., 37 м², 5/26 этаж
Комендантский проспект
улица Лидии Зверевой, 3к3
4 250 000 ₽

2-комн. кв., 45 м², 5/5 этаж
4 290 000 ₽

2-комн. кв., 65 м², 3/25 этаж
Проспект Просвещения
улица Тихоокеанская, 1к1
4 338 000 ₽

1-комн. кв., 42 м², 12/26 этаж
Парнас
6 минут пешком
улица Михаила Дудина, 25к2
4 300 000 ₽

2-комн. кв., 60 м², 4/11 этаж
Звёздная
20 минут на транспорте
4 482 000 ₽

1-комн. кв., 37 м², 2/19 этаж
Звёздная
20 минут пешком
4 574 440 ₽

2-комн. кв., 45 м², 4/9 этаж
Улица Дыбенко
14 минут пешком
улица Шотмана, 6К1
4 650 000 ₽

1-комн. кв., 46 м², 2/17 этаж
Лесна

counter =  333
28
1-комн. кв., 55 м², 5/14 этаж
Площадь Ленина
10 минут на транспорте
улица Жукова, 1
7 547 000 ₽

3-комн. кв., 117 м², 5/12 этаж
Василеостровская
10 минут на транспорте
Уральская улица, 6
15 417 000 ₽

1-комн. кв., 46 м², 3/9 этаж
Чкаловская
5 минут пешком
Корпусная улица, 3
9 350 000 ₽

Студия, 27 м², 7/18 этаж
Рыбацкое
10 минут на транспорте
Садовая улица, 4
1 600 000 ₽

Студия, 25 м², 1/9 этаж
Проспект Ветеранов
30 минут на транспорте
1 689 620 ₽

Студия, 24 м², 5/11 этаж
Звёздная
20 минут на транспорте
1 693 300 ₽

Студия, 25 м², 10/11 этаж
Звёздная
20 минут на транспорте
1 748 600 ₽

Студия, 27 м², 1/3 этаж
Ладожская
8 минут на транспорте
улица Коммуны, 10
1 713 380 ₽

Студия, 19 м², 5/9 этаж
Купчино
20 минут на транспорте
Славянка тер., улица Ростовская
1 793 330 ₽

Студия, 26 м², 9/9 этаж
Проспект Ветеранов
30 минут на транспорте
1 765 840 ₽

Студия, 20 м², 4/9 этаж
Купчино
20 минут на транспорте
Славянка тер., улица Ростовская
1 839 830 ₽

Студия, 26 м², 11/11 

counter =  445
28
1-комн. кв., 37 м², 13/15 этаж
Улица Дыбенко
15 минут пешком
3 434 250 ₽

2-комн. кв., 80 м², 2/5 этаж
Чернышевская
10 минут пешком
улица Некрасова, 37/20
8 990 000 ₽

Квартира в готовом доме!1-комн. кв., 37 м², 8/15 этаж
Улица Дыбенко
15 минут пешком
3 295 000 ₽

Студия, 27 м², 1/11 этаж
Купчино
17 минут на транспорте
2 448 990 ₽

Студия, 24 м², 6/32 этаж
Рыбацкое
15 минут на транспорте
Усть-Славянка тер., улица Заводская
2 442 180 ₽

Студия, 29 м², 11/23 этаж
Пролетарская
15 минут на транспорте
2 401 220 ₽

1-комн. кв., 35 м², 1/9 этаж
Парнас
21 минута на транспорте
2 420 160 ₽

1-комн. кв., 35 м², 10/11 этаж
Звёздная
20 минут на транспорте
2 448 600 ₽

Студия, 27 м², 18/19 этаж
Комендантский проспект
10 минут на транспорте
Комендантский проспект, 57к1
2 438 100 ₽

Студия, 24 м², 15/19 этаж
Комендантский проспект
10 минут на транспорте
Комендантский проспект, 57к1
2 417 000 ₽

Студия, 18 м², 2/5 этаж
Звенигородская
10 минут пешком
Социалистическая улица, 3
2 407 000

...

counter =  1220
28
Теплая светлая однушка на Седова1-комн. кв., 33 м², 3/9 этаж
Пролетарская
20 минут пешком
улица Седова, 109
3 700 000 ₽

5 мин. до м.Черная пешком!4-комн. кв., 171 м², 8/12 этаж
Чёрная речка
2 минуты пешком
Старобельская улица, 4
22 999 000 ₽

1-комн. кв., 39 м², 3/18 этаж
Елизаровская
10 минут пешком
4 878 690 ₽

1-комн. кв., 34 м², 13/19 этаж
Ленинский проспект
7 минут на транспорте
3 790 000 ₽

1-комн. кв., 37 м², 2/12 этаж
Проспект Ветеранов
15 минут на транспорте
3 750 200 ₽

1-комн. кв., 33 м², 1/15 этаж
Проспект Большевиков
11 минут на транспорте
3 750 190 ₽

1-комн. кв., 36 м², 5/6 этаж
3 750 160 ₽

1-комн. кв., 37 м², 2/6 этаж
3 750 120 ₽

1-комн. кв., 45 м², 24/24 этаж
Рыбацкое
15 минут на транспорте
Усть-Славянка тер., улица Заводская
3 767 370 ₽

1-комн. кв., 33 м², 8/20 этаж
Елизаровская
10 минут пешком
3 762 040 ₽

1-комн. кв., 34 м², 13/20 этаж
Елизаровская
10 минут пешком
3 796 250 ₽

1-комн. кв., 38 м², 4/17 этаж
Звёздная
10 минут пешком
3 799 620 ₽


counter =  1329
28
2-комн. кв., 47 м², 6/9 этаж
Гражданский проспект
11 минут пешком
улица Ушинского, 39К1
5 700 000 ₽

1-комн. кв., 38 м², 19/24 этаж
Рыбацкое
15 минут на транспорте
Усть-Славянка тер., улица Заводская
3 700 000 ₽

3-комн. кв., 87 м², 10/11 этаж
Василеостровская
20 минут пешком
24-я Васильевского острова линия, 29
12 614 850 ₽

1-комн. кв., 47 м², 12/15 этаж
Московская
8 минут на транспорте
Пулковское шоссе, 30
3 927 730 ₽

1-комн. кв., 37 м², 1/17 этаж
Площадь Ленина
15 минут на транспорте
3 937 080 ₽

1-комн. кв., 36 м², 9/17 этаж
Площадь Ленина
15 минут на транспорте
3 918 840 ₽

1-комн. кв., 31 м², 1/20 этаж
Елизаровская
10 минут пешком
3 936 600 ₽

1-комн. кв., 44 м², 15/16 этаж
Гражданский проспект
15 минут на транспорте
3 916 000 ₽

1-комн. кв., 33 м², 2/17 этаж
Лесная
15 минут пешком
3 928 190 ₽

1-комн. кв., 37 м², 3/5 этаж
улица Переведенская, 8с2
3 900 000 ₽

1-комн. кв., 39 м², 13/24 этаж
Звёздная
10 минут пешком
3 910 000 ₽

1-комн. кв., 38 м², 13/23 этаж


counter =  1441
28
3-комн. кв., 104 м², 6/10 этаж
Чернышевская
30 минут пешком
18 330 980 ₽

3-комн. кв., 108 м², 5/10 этаж
Площадь Восстания
5 минут пешком
Кременчугская улица, 19к2
12 867 070 ₽

1-комн. кв., 48 м², 2/7 этаж
Технологический институт
15 минут пешком
8 689 040 ₽

1-комн. кв., 40 м², 8/17 этаж
Площадь Ленина
10 минут на транспорте
4 063 370 ₽

1-комн. кв., 35 м², 1/20 этаж
Елизаровская
10 минут пешком
4 099 520 ₽

1-комн. кв., 43 м², 6/19 этаж
Ладожская
8 минут на транспорте
4 061 320 ₽

1-комн. кв., 44 м², 2/19 этаж
Ладожская
8 минут на транспорте
4 087 730 ₽

Студия, 30 м², 7/18 этаж
Елизаровская
10 минут пешком
4 078 250 ₽

1-комн. кв., 37 м², 12/21 этаж
Ладожская
8 минут на транспорте
4 052 100 ₽

1-комн. кв., 31 м², 14/17 этаж
Площадь Ленина
15 минут на транспорте
4 083 330 ₽

2-комн. апарт., 51 м², 9/16 этаж
Площадь Ленина
15 минут на транспорте
4 075 920 ₽

1-комн. кв., 30 м², 2/11 этаж
Чёрная речка
8 минут на транспорте
4 074 310 ₽

1-комн. кв., 37 м², 12/12 этаж

counter =  1553
28
Купить квартиру в центре2-комн. кв., 62 м², 2/8 этаж
Чернышевская
5 минут пешком
Кирочная улица, 31к2
20 700 000 ₽

2-комн. кв., 100 м², 5/7 этаж
Старая Деревня
20 минут пешком
Приморский проспект, 46
20 883 000 ₽

3-комн. кв., 144 м², 6/9 этаж
Чернышевская
15 минут пешком
переулок Басков, 2
33 667 970 ₽

1-комн. кв., 37 м², 16/24 этаж
Комендантский проспект
30 минут пешком
4 243 280 ₽

1-комн. кв., 41 м², 1/17 этаж
Площадь Ленина
15 минут на транспорте
4 241 600 ₽

2-комн. кв., 55 м², 9/9 этаж
Купчино
20 минут на транспорте
Славянка тер., улица Ростовская
4 246 670 ₽

1-комн. кв., 36 м², 4/25 этаж
Звёздная
15 минут пешком
4 223 640 ₽

Студия, 35 м², 4/12 этаж
Комендантский проспект
10 минут на транспорте
4 204 760 ₽

Студия, 27 м², 9/11 этаж
Чёрная речка
8 минут на транспорте
4 204 120 ₽

2-комн. кв., 34 м², 18/23 этаж
Улица Дыбенко
25 минут на транспорте
Октябрьская набережная, 44к2
4 209 220 ₽

1-комн. кв., 39 м², 15/17 этаж
Площадь Ленина
10 минут на транспорте
4

KeyboardInterrupt: 

In [ ]:
cian_url = "https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p=59&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"

# cian_url = next_page(cian_url, cian_soup)
print(cian_url)